In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.tree import export_text
import mglearn
from dashboard_one import *
from dash_model_two import *
from feature_selection import *
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

### sleep, brain_volume_ratio_to_baseline_____VS_____diagnosischanges from every visit


#### sleep_brain_dxch.csv
#### drop column 'NPIKSEV', otherwise we get no samples

In [2]:
com_col = ['Phase', 'RID', 'VISCODE','PTID']
target = 'DXCHANGE'

In [3]:
sleep_brain_dxch = pd.read_csv('sleep_brain_dxch.csv').iloc[:,1:].drop(['NPIKSEV'],axis=1)
sleep_brain_dxch = sleep_brain_dxch[sleep_brain_dxch['DXCHANGE'].notna()].reset_index().drop(['index'],axis=1)   # keep the rows where DXCHANGE is not nan
sleep_brain_dxch

,RID,Phase,VISCODE,PTID,NPIK1,NPIK2,NPIK3,NPIK4,NPIK5,NPIK6,...,ratio_PTAU_bl,Ventricles_reduction_per_year,Hippocampus_reduction_per_year,wholebrain_reduction_per_year,Entorhinal_reduction_per_year,Fusiform_reduction_per_year,ICV_reduction_per_year,ABETA_reduction_per_year,TAU_reduction_per_year,PTAU_reduction_per_year
0,2,ADNI1,m06,011_S_0002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ADNI1,m36,011_S_0002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ADNI1,m60,011_S_0002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,ADNI1,m66,011_S_0002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,ADNI1,m72,011_S_0002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19432,7083,ADNI3,sc,126_S_7083,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19433,7085,ADNI3,sc,941_S_7085,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19434,7088,ADNI3,sc,033_S_7088,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19435,7092,ADNI3,sc,021_S_7092,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### sleep______VS______DXCHANGE

In [4]:
col_lst = [ 'DXCHANGE','NPIK1', 'NPIK2', 'NPIK3', 'NPIK4', 'NPIK5', 'NPIK6', 'NPIK7', 'NPIK8',
       'NPIK9A', 'NPIK9B', 'NPIK9C', 'NPIKTOT',  'insomnia','OSA']
sleep_dxch = sleep_brain_dxch[com_col + col_lst].set_index(['Phase', 'RID', 'VISCODE','PTID']).dropna(how='any',axis=0).reset_index()
sleep_dxch

,Phase,RID,VISCODE,PTID,DXCHANGE,NPIK1,NPIK2,NPIK3,NPIK4,NPIK5,NPIK6,NPIK7,NPIK8,NPIK9A,NPIK9B,NPIK9C,NPIKTOT,insomnia,OSA
0,ADNI2,8,v06,011_S_0008,CN-CN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,6.0,1.0,0.0
1,ADNI2,8,v41,011_S_0008,CN-MCI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,1.0,8.0,1.0,0.0
2,ADNI2,31,v06,023_S_0031,CN-CN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,0.0
3,ADNI2,31,v11,023_S_0031,CN-CN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,0.0
4,ADNI2,31,v21,023_S_0031,CN-CN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,ADNI3,6890,y1,021_S_6890,MCI-MCI,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0
903,ADNI3,6891,y1,123_S_6891,AD-AD,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,2.0,3.0,8.0,1.0,0.0
904,ADNI3,6897,y1,036_S_6897,MCI-MCI,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
905,ADNI3,6947,y1,035_S_6947,MCI-MCI,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


In [5]:
np.sum(sleep_dxch.isna())   # check nan

Phase       0
RID         0
VISCODE     0
PTID        0
DXCHANGE    0
NPIK1       0
NPIK2       0
NPIK3       0
NPIK4       0
NPIK5       0
NPIK6       0
NPIK7       0
NPIK8       0
NPIK9A      0
NPIK9B      0
NPIK9C      0
NPIKTOT     0
insomnia    0
OSA         0
dtype: int64

In [6]:
sleep_dxch.groupby('DXCHANGE').count()

,Phase,RID,VISCODE,PTID,NPIK1,NPIK2,NPIK3,NPIK4,NPIK5,NPIK6,NPIK7,NPIK8,NPIK9A,NPIK9B,NPIK9C,NPIKTOT,insomnia,OSA
DXCHANGE,,,,,,,,,,,,,,,,,,
AD-AD,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146,146
AD-MCI,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
CN-AD,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
CN-CN,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201
CN-MCI,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
MCI-AD,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77
MCI-CN,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41
MCI-MCI,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410


- select only the MCI-AD, MCI-MCI

In [7]:
sleep_dxch_22g = sleep_dxch.loc[sleep_dxch['DXCHANGE'].isin(['CN-CN','CN-MCI'])].reset_index().drop(['index'],axis=1)
sleep_dxch_22g.shape  # model not good

(229, 19)

In [8]:
sleep_dxch_2g = sleep_dxch.loc[sleep_dxch['DXCHANGE'].isin(['MCI-AD','MCI-MCI'])].reset_index().drop(['index'],axis=1)
sleep_dxch_2g.shape

(487, 19)

In [9]:
sleep_dxch_3g = sleep_dxch.loc[sleep_dxch['DXCHANGE'].isin(['MCI-AD','MCI-MCI','MCI-CN'])].reset_index().drop(['index'],axis=1)
sleep_dxch_3g.shape

(528, 19)

### oversampling and undersampling

- functions
    - models(df,drop_lst,target) : under sampling, split, scale, pca, models
    - cv_models(df,drop_lst,target,k): under sampling, NOT SPLIT, scale, pca, models with cross-validation 

In [10]:
drop_lst = ['Phase', 'RID', 'VISCODE', 'PTID','DXCHANGE']
k = 10

- 'MCI-AD': 77, 'MCI-MCI': 77
- 10 pca components, 20trees. average weighted f1-score of 10-cross validation:0.622

In [12]:
cv_models(sleep_dxch_2g,drop_lst,target,k)

After undersampling data size is 154 ; Resampled dataset shape Counter({'MCI-AD': 77, 'MCI-MCI': 77})

10 principle components are needed to explain 90% of the data

- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.490
          - saga_L1, average weighted f1-score of 10-cross validation:0.349
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.490
       - C = 0.01
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.534
          - saga_L1, average weighted f1-score of 10-cross validation:0.341
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.534
       - C = 0.1
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.591
          - saga_L1, average weighted f1-score of 10-cross validation:0.483
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.591
       - C = 1
          -

          - 25trees. average weighted f1-score of 10-cross validation:0.514
          - 30trees. average weighted f1-score of 10-cross validation:0.523
          - 35trees. average weighted f1-score of 10-cross validation:0.515
          - 40trees. average weighted f1-score of 10-cross validation:0.510
          - 45trees. average weighted f1-score of 10-cross validation:0.523
          - 50trees. average weighted f1-score of 10-cross validation:0.508
          - 55trees. average weighted f1-score of 10-cross validation:0.514
          - 60trees. average weighted f1-score of 10-cross validation:0.514
          - 65trees. average weighted f1-score of 10-cross validation:0.507
          - 70trees. average weighted f1-score of 10-cross validation:0.514
          - 75trees. average weighted f1-score of 10-cross validation:0.507
          - 80trees. average weighted f1-score of 10-cross validation:0.508
          - 85trees. average weighted f1-score of 10-cross validation:0.508
          - 

In [13]:
cv_models(sleep_dxch_3g,drop_lst,target,k)

After undersampling data size is 123 ; Resampled dataset shape Counter({'MCI-AD': 41, 'MCI-CN': 41, 'MCI-MCI': 41})

9 principle components are needed to explain 90% of the data

- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.433
          - saga_L1, average weighted f1-score of 10-cross validation:0.167
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.433
       - C = 0.01
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.476
          - saga_L1, average weighted f1-score of 10-cross validation:0.160
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.476
       - C = 0.1
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.440
          - saga_L1, average weighted f1-score of 10-cross validation:0.419
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.440
       - C = 

C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.442
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.442
       - C = 100
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.431


C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

          - saga_L1, average weighted f1-score of 10-cross validation:0.431
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.431
       - C = 1000
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.421


...

          - saga_L1, average weighted f1-score of 10-cross validation:0.421
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.421
    - Decision tree
          - tree depth: 1.000. average weighted f1-score of 10-cross validation:0.398
          - tree depth: 2.000. average weighted f1-score of 10-cross validation:0.421
          - tree depth: 3.000. average weighted f1-score of 10-cross validation:0.386
          - tree depth: 4.000. average weighted f1-score of 10-cross validation:0.391
          - tree depth: 5.000. average weighted f1-score of 10-cross validation:0.349
          - tree depth: 6.000. average weighted f1-score of 10-cross validation:0.332
          - tree depth: 7.000. average weighted f1-score of 10-cross validation:0.347
          - tree depth: 8.000. average weighted f1-score of 10-cross validation:0.361
          - tree depth: 9.000. average weighted f1-score of 10-cross validation:0.359
          - tree depth: 10.000. average weighted 

C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.437
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.437
       - C = 1000
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.437


C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

          - saga_L1, average weighted f1-score of 10-cross validation:0.451
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.437
    - Decision tree
          - tree depth: 1.000. average weighted f1-score of 10-cross validation:0.345
          - tree depth: 2.000. average weighted f1-score of 10-cross validation:0.415
          - tree depth: 3.000. average weighted f1-score of 10-cross validation:0.388
          - tree depth: 4.000. average weighted f1-score of 10-cross validation:0.375
          - tree depth: 5.000. average weighted f1-score of 10-cross validation:0.290
          - tree depth: 6.000. average weighted f1-score of 10-cross validation:0.348
          - tree depth: 7.000. average weighted f1-score of 10-cross validation:0.348
          - tree depth: 8.000. average weighted f1-score of 10-cross validation:0.342
          - tree depth: 9.000. average weighted f1-score of 10-cross validation:0.317
          - tree depth: 10.000. average weighted 